火災情報のヒートマップ表示
=====================

NASAが公開している "火災情報"(https://firms.modaps.eosdis.nasa.gov/active_fire/c6/text/MODIS_C6_Russia_and_Asia_24h.csv) を読みこみ、ヒートマップで表示してみます。

ライブラリの読み込み
-----------------
pandasは一般的にデータ解析に使用されるライブラリで、一般的にpdと略してimportします。  
また、web上からcsvファイルをダウンロードして来て取り込む為、ioおよびrequestsもimportします。

すべてのライブラリはdatalabに標準で組み込まれています。

In [76]:
import pandas as pd
import io
import requests

読み込み処理はrequestsを使ってgetし、contentをutf-8にdecodeして、read_csvで読み込みます。  
decodeに渡す文字コードはcsvファイルの文字コードにあわせて変更します。

decodeの結果はDataFrame型で帰ります。

In [77]:
url = "https://firms.modaps.eosdis.nasa.gov/active_fire/c6/text/MODIS_C6_Russia_and_Asia_24h.csv"
modis = pd.read_csv(io.StringIO(requests.get(url).content.decode('utf-8')))

type(modis)

pandas.core.frame.DataFrame

最初の5行を確認してみます。

In [78]:
modis[:5]

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,confidence,version,bright_t31,frp,daynight
0,25.062,121.629,321.9,1.0,1.0,2018-04-13,245,T,32,6.0NRT,300.5,10.4,D
1,24.931,121.167,322.1,1.0,1.0,2018-04-13,245,T,56,6.0NRT,300.9,9.9,D
2,24.191,120.815,317.9,1.0,1.0,2018-04-13,245,T,26,6.0NRT,300.7,8.3,D
3,23.963,120.974,319.8,1.0,1.0,2018-04-13,245,T,52,6.0NRT,303.0,8.3,D
4,23.017,120.341,316.7,1.0,1.0,2018-04-13,245,T,30,6.0NRT,297.6,5.1,D


DataFrame型はlocで部分データを切り取ることが出来ます。行、列の順で切りとる範囲を指定します。

In [79]:
brightness = modis.loc[:10, ['latitude', 'longitude', 'brightness']].rename(columns={'latitude':'lat', 'longitude':'lng', 'brightness':'weight'})
brightness

,lat,lng,weight
0,25.062,121.629,321.9
1,24.931,121.167,322.1
2,24.191,120.815,317.9
3,23.963,120.974,319.8
4,23.017,120.341,316.7
5,23.015,120.351,323.0
6,22.544,120.358,327.2
7,17.871,121.250,314.2
8,17.490,121.644,335.0
9,17.426,121.840,324.5


chartコマンドのhelpを出して、可視化方法を確認してみます。

In [80]:
%%chart --help

usage: %%chart [-h]
               {annotation,area,bars,bubbles,calendar,candlestick,columns,combo,gauge,geo,heatmap,histogram,line,map,org,paged_table,pie,sankey,scatter,stepped_area,table,timeline,treemap}
               ...

Generate an inline chart using Google Charts using the data in a Table, Query,
dataframe, or list. Numerous types of charts are supported. Options for the
charts can be specified in the cell body using YAML or JSON.

positional arguments:
  {annotation,area,bars,bubbles,calendar,candlestick,columns,combo,gauge,geo,heatmap,histogram,line,map,org,paged_table,pie,sankey,scatter,stepped_area,table,timeline,treemap}
                        commands
    annotation          Generate a annotation chart.
    area                Generate a area chart.
    bars                Generate a bars chart.
    bubbles             Generate a bubbles chart.
    calendar            Generate a calendar chart.
    candlestick         Generate a candlestick chart.
    columns          

地図上に重ねて情報を表示するのはmapなので、さらにmapのヘルプを確認します。

In [81]:
%%chart map -h

usage: %%chart map [-h] [-f FIELDS] -d DATA

Generate a map chart.

optional arguments:
  -h, --help            show this help message and exit
  -f FIELDS, --fields FIELDS
                        The field(s) to include in the chart
  -d DATA, --data DATA  The name of the variable referencing the Table or
                        Query to chart
None


-fで表示対象のフィールドを、-dで表示するデータの変数名を渡し、さらに本文でJSONまたはYAML形式で設定出来るとの事なので、google map APIを参考にパラメータを渡してみます。

In [82]:
%%chart map -f lat,lng,weight -d brightness
mapType: normal

でもなぜか出ない。